In [30]:
import pandas
import numpy
import pprint

import cmapPy.pandasGEXpress.parse as parse
import cmapPy.pandasGEXpress.GCToo as GCToo
import cmapPy.pandasGEXpress.write_gct as write_gct
import cmapPy.pandasGEXpress.write_gctx as write_gctx

import cmapPy.pandasGEXpress.subset_gctoo as subset_gctoo

In [20]:
input_query_file = "/Users/dlahr/temp/done/rnaseq_SP_NS-22.0001/analysis/count_data/NS-22.0001_log2_TPM_r57771x86.gct"
input_ref_file = "fsd_cache/Achilles/2022_Q2_CCLE_expression_r19221x1406.gctx"

In [34]:
output_template = "test_cell_line_auth_{}_data_r{}x{}"

In [21]:
query_rm_df = parse.parse(input_query_file, row_meta_only=True)
display(query_rm_df)

rhd,gene_symbol
rid,
ENSG00000223972,DDX11L1
ENSG00000227232,WASH7P
ENSG00000278267,MIR6859-1
ENSG00000243485,MIR1302-2
ENSG00000284332,NaN
...,...
ENSG00000276017,AC007325.1
ENSG00000278817,AC007325.4
ENSG00000277196,AC007325.2


In [4]:
ref_rm_df = parse.parse(input_ref_file, row_meta_only=True)
display(ref_rm_df)

rhd,entrez_id
rid,
TSPAN6,7105
TNMD,64102
DPM1,8813
SCYL3,57147
C1orf112,55732
...,...
ELOA3B,728929
NPBWR1,2831
ELOA3D,100506888


In [22]:
t = set(ref_rm_df.index)
u = set(query_rm_df.gene_symbol)

ref_only_gene_set = t - u
print(len(ref_only_gene_set))

query_only_gene_set = u - t
print(len(query_only_gene_set))

intersection_gene_set = u & t
print(len(intersection_gene_set))

816
37067
18405


In [23]:
chosen_ref_genes = numpy.random.choice(list(ref_only_gene_set), size=11, replace=False)
print(chosen_ref_genes)
chosen_query_genes = numpy.random.choice(list(query_only_gene_set), size=13, replace=False)
print(chosen_query_genes)
chosen_intersection_genes = numpy.random.choice(list(intersection_gene_set), size=97, replace=False)
print(chosen_intersection_genes)

['DEPP1' 'MUCL3' 'REELD1' 'YAE1' 'H2BC7' 'TLCD3B' 'CEMIP2' 'FAM243A'
 'PACC1' 'H1-7' 'TLCD3A']
['MIR4791' 'AC114776.3' 'ZEB2P1' 'RP11-480G7.3' 'LINC01249' 'AC116609.1'
 'RP11-139K4.2' 'RP11-358H9.1' 'CTD-2078B5.2' 'RP11-950C14.7'
 'CTB-51J22.1' 'RP11-451H23.2' 'AC093375.1']
['IQCH' 'ADGRV1' 'ID1' 'ALOX12' 'UCP3' 'LUZP1' 'PRR20E' 'PCYT1B' 'CGGBP1'
 'PCSK1' 'SLC25A40' 'SPRR2B' 'ATG4D' 'CREB3' 'TMPRSS11D' 'FAM25C' 'BDH2'
 'C14orf119' 'CXCR1' 'KRTAP27-1' 'CDC14A' 'APEX2' 'NR4A2' 'EPHX3' 'PSMB7'
 'MYT1L' 'RASA4B' 'ZSWIM7' 'GPBP1' 'CBLN3' 'PCDHB7' 'ZNF608' 'FSIP1'
 'GRIA2' 'ALDH6A1' 'PISD' 'PHF14' 'NAT8' 'CCDC3' 'NEU3' 'UBALD1' 'CDY1B'
 'PIGS' 'REV3L' 'RRP15' 'CSF1' 'TTLL1' 'MBD3L3' 'LRRC40' 'PGBD5'
 'ANKRD34A' 'SCGB1D1' 'LARGE1' 'TMEM70' 'STX3' 'DCDC2C' 'PON2' 'KPNA1'
 'CCDC65' 'AGR2' 'TSPAN19' 'SLC7A2' 'CENPVL2' 'SEC61B' 'DDX11' 'MNDA'
 'PUS1' 'PDX1' 'SKP1' 'LAMA2' 'QRICH1' 'CCL11' 'RAI14' 'KDM2A' 'FHL3'
 'OSM' 'TMEM94' 'KIAA0319' 'MPZ' 'TBC1D19' 'NEK2' 'CHCHD6' 'NDST3'
 'ARHGEF26' 'WNT6' 

In [24]:
new_ref_genes = list(chosen_ref_genes) + list(chosen_intersection_genes)
new_ref_genes.sort()
print(len(new_ref_genes))

new_query_genes = list(chosen_query_genes) + list(chosen_intersection_genes)
new_query_genes.sort()
print(len(new_query_genes))

108
110


In [25]:
locs = query_rm_df.gene_symbol.isin(set(new_query_genes))
assert locs.sum() == len(new_query_genes)
new_query_ens_ids = query_rm_df.index[locs].to_list()
print(len(new_query_ens_ids))

110


In [26]:
query_g = parse.parse(input_query_file, rid=new_query_ens_ids)
print(query_g)

GCT1.3
src: /Users/dlahr/temp/done/rnaseq_SP_NS-22.0001/analysis/count_data/NS-22.0001_log2_TPM_r57771x86.gct
data_df: [110 rows x 86 columns]
row_metadata_df: [110 rows x 1 columns]
col_metadata_df: [86 rows x 32 columns]


In [32]:
print(query_g.col_metadata_df.bio_context_id.unique())
depmap_id_lookup_dict = {"22RV1":"ACH-000956", "PC3":"ACH-000090", "SYO1":"ACH-001275"}
pprint.pprint(depmap_id_lookup_dict)
query_g.col_metadata_df["DepMap_ID"] = [depmap_id_lookup_dict[x] for x in query_g.col_metadata_df.bio_context_id]
query_g.col_metadata_df

['22RV1' 'PC3' 'SYO1']
{'22RV1': 'ACH-000956', 'PC3': 'ACH-000090', 'SYO1': 'ACH-001275'}


chd,assay,bio_sample_type,donor_id,in_vivo_animal_id,in_vivo_tissue_type,bio_context_parental_id,bio_context_id,bio_context_modification,special_media_for_bio_context,pert_id,...,sample_prep_contacts,sample_prep_ELN_entry_URL,ChIP.seq.antibody,comments,pert_dose_str,pert_time_str,group,experiment_id,R_group,DepMap_ID
cid,,,,,,,,,,,,,,,,,,,,,
A1,RNA-seq,cell line (in vitro),NaN,NaN,NaN,22RV1,22RV1,NaN,NaN,DMSO,...,Salonee,NaN,NaN,NaN,NaN,8 h,22RV1_DMSO_8,NS-22.0001,x_22RV1_DMSO_8,ACH-000956
A2,RNA-seq,cell line (in vitro),NaN,NaN,NaN,22RV1,22RV1,NaN,NaN,DMSO,...,Salonee,NaN,NaN,NaN,NaN,8 h,22RV1_DMSO_8,NS-22.0001,x_22RV1_DMSO_8,ACH-000956
A3,RNA-seq,cell line (in vitro),NaN,NaN,NaN,22RV1,22RV1,NaN,NaN,DMSO,...,Salonee,NaN,NaN,NaN,NaN,8 h,22RV1_DMSO_8,NS-22.0001,x_22RV1_DMSO_8,ACH-000956
A4,RNA-seq,cell line (in vitro),NaN,NaN,NaN,22RV1,22RV1,NaN,NaN,FHT-0006394,...,Salonee,NaN,NaN,NaN,15 nM,8 h,22RV1_FHT0006394_8,NS-22.0001,x_22RV1_FHT0006394_8,ACH-000956
A5,RNA-seq,cell line (in vitro),NaN,NaN,NaN,22RV1,22RV1,NaN,NaN,FHT-0006394,...,Salonee,NaN,NaN,NaN,15 nM,8 h,22RV1_FHT0006394_8,NS-22.0001,x_22RV1_FHT0006394_8,ACH-000956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C17,RNA-seq,cell line (in vitro),NaN,NaN,NaN,SYO1,SYO1,NaN,NaN,FHT-0006394,...,Salonee,NaN,NaN,NaN,15 nM,24 h,SYO1_FHT0006394_24,NS-22.0001,SYO1_FHT0006394_24,ACH-001275
C18,RNA-seq,cell line (in vitro),NaN,NaN,NaN,SYO1,SYO1,NaN,NaN,FHT-0006394,...,Salonee,NaN,NaN,NaN,15 nM,24 h,SYO1_FHT0006394_24,NS-22.0001,SYO1_FHT0006394_24,ACH-001275
C19,RNA-seq,cell line (in vitro),NaN,NaN,NaN,SYO1,SYO1,NaN,NaN,FHT-0068291,...,Salonee,NaN,NaN,NaN,100 nM,24 h,SYO1_FHT0068291_24,NS-22.0001,SYO1_FHT0068291_24,ACH-001275


In [33]:
ref_g = parse.parse(input_ref_file, rid=new_ref_genes)
print(ref_g)

b'GCTX1.0'
src: fsd_cache/Achilles/2022_Q2_CCLE_expression_r19221x1406.gctx
data_df: [108 rows x 1406 columns]
row_metadata_df: [108 rows x 1 columns]
col_metadata_df: [1406 rows x 0 columns]


In [36]:
output_file = output_template.format("query", query_g.data_df.shape[0], query_g.data_df.shape[1])
print(output_file)
write_gct.write(query_g, output_file)
write_gctx.write(query_g, output_file)

test_cell_line_auth_query_data_r110x86


In [38]:
output_file = output_template.format("ref", ref_g.data_df.shape[0], ref_g.data_df.shape[1])
print(output_file)
write_gct.write(ref_g, output_file)
write_gctx.write(ref_g, output_file)

test_cell_line_auth_ref_data_r108x1406
